In [ ]:
!pip install huggingface_hub transformers datasets langchain openai chromadb sentencepiece ctransformers -q

In [ ]:
!pip install faiss-cpu

In [1]:
import pandas as pd
import numpy as np

## data prep

In [85]:
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/NLP_RAG_Open_Source/intent_complete_22_nov_2023.csv").dropna()

In [86]:
df.head()

,prompt,completion
0,"[{""role"": ""user"", ""content"": ""You are to class...",provide_concept
1,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_answer
2,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill
3,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill
4,"[{""role"": ""user"", ""content"": ""You are to class...",diagnosis_result\nexplain_drill


In [83]:
def get_latest_user_input_from_prompt(prompt:str):
  input = prompt.split("Here is the message you are to classify:")
  if "Here are previous chats for your reference (only use this if you need further information to infer the intent):" in prompt:
    input = input[1].split("Here are previous chats for your reference (only use this if you need further information to infer the intent):")
  else:
    input = input[1].split("Here are previous chats or summary conversation for your reference (only use this if you need further information to infer the intent)")
  return input[0]

In [93]:
df['user_message'] = df['prompt'].apply(get_latest_user_input_from_prompt).replace(r"[\\n]", "", regex=True)

In [94]:
df

,prompt,completion,user_message
0,"[{""role"": ""user"", ""content"": ""You are to class...",provide_concept,Cotoh cotoh huruf voiced soud
1,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_answer,aku salah semua
2,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill,u25cb-o geogebra.org /m/cjdyKGes GebraCircumfe...
3,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill,Kita bahas teses aja
4,"[{""role"": ""user"", ""content"": ""You are to class...",diagnosis_result\nexplain_drill,Lihat hasil diagosisBahas soal omor 1
...,...,...,...
8095,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_with_topic,belum
8096,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_with_topic,deret aritmatika
8097,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_with_topic,itegral
8098,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_with_topic,fugsi aljabar


In [96]:
intents = ['homework',
 'homework_with_question',
 'explain_answer',
 'explain_answer_with_topic',
 'core_explanation_repeat',
 'core_explanation_exercise',
 'core_explanation_video',
 'core_explanation_protips',
 'negative_respond',
 'positive_respond',
 'recommend',
 'recommend_with_topic',
 'assess_knowledge',
 'assess_knowledge_with_topic',
 'assess_knowledge_answer',
 'exercise',
 'exercise_with_image',
 'exercise_with_topic',
 'exercise_multiple_question',
 'exercise_multiple_question_with_topic',
 'personal',
 'general_intent',
 'provide_topic',
 'provide_subject',
 'provide_subtopic',
 'provide_concept',
 'finish_learning',
 'back_to_study',
 'similar_question',
 'provide_answer',
 'diagnosis_result',
 'explain_drill']

In [97]:
df_clean = df[df['completion'].isin(intents)].reset_index(drop=True).rename(columns={
    'completion' : 'intent'
})

In [98]:
df_clean.head()

,prompt,intent,user_message
0,"[{""role"": ""user"", ""content"": ""You are to class...",provide_concept,Cotoh cotoh huruf voiced soud
1,"[{""role"": ""user"", ""content"": ""You are to class...",assess_knowledge_answer,aku salah semua
2,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill,u25cb-o geogebra.org /m/cjdyKGes GebraCircumfe...
3,"[{""role"": ""user"", ""content"": ""You are to class...",explain_drill,Kita bahas teses aja
4,"[{""role"": ""user"", ""content"": ""You are to class...",exercise_multiple_question_with_topic,1. Sebutka peyimpaga2 yag terjadi pada masa De...


In [105]:
data = df_clean['user_message'].tolist()[:10]

In [106]:
data

['Cotoh cotoh huruf voiced soud',
 'aku salah semua',
 'u25cb-o geogebra.org /m/cjdyKGes GebraCircumferece of CirclesAuthor: Athoy OR u67efu5fd7u660eTopic: Circle(Seehttp://www.geogebratube.org/material/show/id/8222 for aotherew versio.)Drag the gree poit to appropriate positio to measure the diameter of the circle.Drag the red poit P to straighte the circumferece. Divide the scale of the ruler to measure its legth to two decimal places. Use the Zoom I, Zoom Out ad Move tools if ecessary.Calculate the values of c/d. What do you discover?divide the scale[I]maksudya gimaa si?',
 'Kita bahas teses aja',
 '1. Sebutka peyimpaga2 yag terjadi pada masa Demokrasi Terpimpi2. Deskripsika Gagasa Maipol-USDEK Soekaro3 Deskripsika ttg ajara Nasakom da bagaimaa Dampakya thdp PKI pada masa Demokrasi Terpimpi4.Sebutka Kabiet-kabiet pada masa demokrasi Terpimpi beserta Programya5. Sebutka da Jelaska kebijaka-kebijaka Politik da Ekoomi pada masa Domokrasi Terpimpi dilihat dari : a. Latar belakagb. Lagka

## embedddings

In [117]:
from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [118]:
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DataFrameLoader

In [119]:
from langchain.embeddings import HuggingFaceHubEmbeddings

In [141]:
embeddings = HuggingFaceHubEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [142]:
loader = DataFrameLoader(df_clean, page_content_column="user_message")
data = loader.load()

In [143]:
df_clean.shape

(6996, 3)

In [144]:
len(data)

6996

In [145]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

In [146]:
# db = FAISS.from_documents(all_splits, embeddings)
db = Chroma.from_documents(documents=all_splits, embedding=embeddings)

KeyError: ignored

In [ ]:
query = "aku mau belajar stoikiometri"
docs = db.similarity_search(query, k=4)

In [76]:
docs

[Document(page_content='aku butuh bantuan soal belajar teknologi nuklir', metadata={'intent': 'homework_with_question', 'state': 'HomeworkState'}),
 Document(page_content='Bantu aku ngerjain tugas IPA ku', metadata={'intent': 'homework', 'state': 'HomeworkState'}),
 Document(page_content='jadwal belajar untuk unjian sekolah', metadata={'intent': 'general_intent', 'state': 'GeneralState'}),
 Document(page_content='jurnal penyesuaian akuntansi', metadata={'intent': 'provide_topic', 'state': 'GeneralState'})]